In [ ]:
pip install mysql.connector

In [ ]:
pip install datetime

In [3]:
import requests 
import mysql.connector
from datetime import datetime


API_KEY = #Your API key
REGION = 'IN'
MAX_RESULT = 10

conn=mysql.connector.connect(
    host = #your host name,
    user = #your user,
    passwd = #your password,
    database = #database name
)

cursor = conn.cursor()

cursor.execute("DELETE FROM yt_trending")
cursor.execute("ALTER TABLE yt_trending AUTO_INCREMENT = 1")
conn.commit()


url = f"https://www.googleapis.com/youtube/v3/videos?part=snippet,statistics&chart=mostPopular&regionCode={REGION}&maxResults={MAX_RESULT}&key={API_KEY}"
response = requests.get(url)
data = response.json()

for item in data.get('items',[]):
    video_id = item['id']
    title = item['snippet']['title']
    yt_channel = item['snippet']['channelTitle']
    views = int(item['statistics'].get('viewCount', 0))
    published_at = item['snippet']['publishedAt']
    published_dt = datetime.strptime(published_at, "%Y-%m-%dT%H:%M:%SZ")

    # Insert into MySQL
    cursor.execute("""
        INSERT IGNORE INTO yt_trending (video_id, title, yt_channel, views, published_at)
        VALUES (%s, %s, %s, %s, %s)
    """, (video_id, title, yt_channel, views, published_dt))

conn.commit()
cursor.close()
conn.close()

print("✅ Top 10 trending videos stored in MYSQL.")

✅ Top 10 trending videos stored in MYSQL.
